<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/LSTM_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span> NLP : Entrainnement et sauvegarde du modèle LSTM/GRU </span>
<hr style="border-bottom: solid;background-color:light;color:black;">

* [Installations](#section-1)
* [Imports](#section-2)
* [Choix des paramètres](#section-3)
* [Lecture des données](#section-4)
* [Preprocessing](#section-5)
* [Création du modèle](#section-6)
* [Entrainement du modèle](#section-7)
* [Prédiction des données](#section-8)
* [Sauvegarde du modèle](#section-9)

<a name="section-1"></a>
# <span>1. Installation des packages</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [ ]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

Tue Dec 27 09:56:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

<a name="section-2"></a>
# <span>2. Imports </span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [ ]:
# basics 
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle

# tensorflow
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.layers import Conv1D, MaxPooling1D

# plot 
import matplotlib.pyplot as plt 
import seaborn as sns 

# nltk 
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


<a name="section-3"></a>
# <span>3. Choix des paramètres</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

<p align="justify">Dans cette section, nous pouvons définir les paramètres du modèles LSTM. Comme paramètres d'entrés, on retrouve la taille des séquences, la taille des données d'entrainement et de validations. Il est également possible de définir des paramètres d'entrainements du modèle. Parmis ceux-ci, on retrouve le batch size ainsi que le nombre d'epochs. Le path des données d'entrainement, ajoutées et de tests est aussi défini dans cette section.</p>

In [ ]:
MAX_SEQ_LEN = 5000
DONNEE_AJOUTEES = 200
TRAIN_SIZE = 0.8

batch_size = 64
num_epochs = 3

train_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_train.csv'
added_path = '/content/drive/MyDrive/HandOnAI_2_NLP/added_train.csv'
test_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_test.csv'

<a name="section-4"></a>
# <span>4. Lecture des données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">
<p align="justify">La section suivante consiste simplement à récupérer les donnéeset les stocker dans des dataframes à l'aide de la librairie pandas.</p>

In [ ]:
df = pd.read_csv(train_path)
df_added = pd.read_csv(added_path)
df_test = pd.read_csv(test_path)

<a name="section-5"></a>
# <span>5. Preprocessing</span>
<hr style="border-bottom: solid;background-color:light;color:black;">
<p align="justify">La partie preprocessing est composé de deux parties. Dans la première étape, appliquons un nettoyage de données. Les caractères spéciaux ainsi que les stopwords sont supprimés des données d'entrainnement et de test. Les données abérants trouvées lors de l'analyse de données sont également enlevées. Durant la seconde étape, les données sont tokenisées. </p>

## <span>5.1 Nettoyage de données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [ ]:
df = df.drop(['Unnamed: 0', 'target_name'], axis = 1)
df_added.rename(columns = {'french':'data'}, inplace = True)
df_added = df_added.drop(['Unnamed: 0'], axis = 1)
df_test = df_test.drop(['Unnamed: 0', 'target_name'], axis = 1)

df = df.append(df_added[:DONNEE_AJOUTEES], ignore_index=True)

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
STOPWORDS = set(stopwords.words('french'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [ ]:
df['data'] = df['data'].apply(clean_text)
df_test['data'] = df_test['data'].apply(clean_text)

df = df.drop(df.index[1430])
df = df.drop(df.index[1429])
df = df.drop(df.index[1180])
df = df.drop(df.index[1136])
     
df = df.reset_index()


## <span>5.2 Tokenisation des données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [ ]:
df_train, df_val = train_test_split(df, test_size=0.2, shuffle=True)

X_train = df_train.data.tolist()
X_val = df_val.data.tolist()
X_test = df_test.data.tolist()

y_train = df_train.label.tolist()
y_val = df_val.label.tolist()
y_test = df_test.label.tolist()

train_text_vec = [text for text in X_train]
val_text_vec = [text for text in X_val]
test_text_vec = [text for text in X_test]

# tokenize the sentences
tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(train_text_vec)

train_text_vec = tokenizer.texts_to_sequences(train_text_vec)
val_text_vec = tokenizer.texts_to_sequences(val_text_vec)
test_text_vec = tokenizer.texts_to_sequences(test_text_vec)

# pad the sequences
train_text_vec = pad_sequences(train_text_vec, maxlen=MAX_SEQ_LEN)
val_text_vec = pad_sequences(val_text_vec, maxlen=MAX_SEQ_LEN)
test_text_vec = pad_sequences(test_text_vec, maxlen=MAX_SEQ_LEN)


In [ ]:
# One Hot Encode Y values:
encoder = LabelEncoder()

y_train = encoder.fit_transform(df_train['label'].values)
y_train = to_categorical(y_train) 

y_val = encoder.fit_transform(df_val['label'].values)
y_val = to_categorical(y_val) 

y_test = encoder.fit_transform(df_test['label'].values)
y_test = to_categorical(y_test) 


<a name="section-6"></a>
# <span>6. Création du modèle</span>
<hr style="border-bottom: solid;background-color:light;color:black;">
<p align="justify">Pour la construction du modèle, nous appliquons en amont une couvche convolutionnelle afin d'extraire des caractéristiques. Suite à l'extraction de caractériques, nous appliquons plusieurs couches LSTM. Ces couches sont bidirectionnelles afin d'avoir un maximum d'informations. Le modèle se termine avec de simples couches afin d'effectuer la classification. </p>

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = (len(tokenizer.word_counts) + 1), output_dim = 128, input_length = MAX_SEQ_LEN))
model.add(Conv1D(32,3, padding='same'))
model.add(MaxPooling1D())
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

<a name="section-7"></a>
# <span>7. Entrainement du modèle</span>
<hr style="border-bottom: solid;background-color:light;color:black;">
<p align="justify">Maintenant que l'architecture est terminée, l'entrainement peut commencer. Comme loss nous avons choisit le crossentropy binaire avec l'optimizer adam. </p>

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_text_vec, y_train, validation_data=(val_text_vec, y_val), batch_size=batch_size, epochs=num_epochs)

Epoch 1/3
21/21 [==============================] - 26s 586ms/step - loss: 0.6326 - accuracy: 0.6183 - val_loss: 0.3875 - val_accuracy: 0.8489
Epoch 2/3
21/21 [==============================] - 11s 503ms/step - loss: 0.2524 - accuracy: 0.9176 - val_loss: 0.3299 - val_accuracy: 0.8761
Epoch 3/3
21/21 [==============================] - 11s 504ms/step - loss: 0.0464 - accuracy: 0.9894 - val_loss: 0.3566 - val_accuracy: 0.9215


<a name="section-8"></a>
# <span>8. Évaluation du modèle</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [ ]:
test_scores = model.evaluate(test_text_vec, y_test, verbose=1)
print("test scores:", test_scores)

16/16 [==============================] - 2s 146ms/step - loss: 0.2086 - accuracy: 0.9568
test scores: [0.20857766270637512, 0.9567901492118835]


<a name="section-9"></a>
# <span>9. Sauvegarde du modèle et du tokenizer</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [ ]:
model.save('/content/drive/MyDrive/HandOnAI_2_NLP/LSTM_model.h5')

In [ ]:
with open('/content/drive/MyDrive/HandOnAI_2_NLP/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)